In [1]:
import pandas as pd
import cv2
import numpy as np

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
import torch.utils.data as data
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ExponentialLR

import pickle
from tqdm.notebook import tqdm


In [2]:
def preprocess_data(directory:str, batch_size:int, test_size:int, rand_num:int, worker:int):
    '''
        directory: the directory of processed directory with class folders inside
        batch_size: size of batch for training
        test_size: percent of dataset used for test
        rand_num: put random number for reproducibility
        worker: number of worker in computation
        
        return train and test data ready for training
    '''
    #pipeline to resize images, crop, convert to tensor, and normalize
    trans = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])
    
    dataset = torchvision.datasets.ImageFolder(root=directory, transform=trans) #read image in folder to data with labels
    
    train_len = len(dataset) #get length of whole data
    ind = list(range(train_len)) #indices of whole data
    spl = int(np.floor(test_size * train_len)) #index of test data
    
    #reproducibility and shuffle step
    np.random.seed(rand_num) 
    np.random.shuffle(ind)
    
    #sampling preparation steps
    train_id, test_id = ind[spl:], ind[:spl]
    tr_sampl = SubsetRandomSampler(train_id)
    te_sampl = SubsetRandomSampler(test_id)

    #use data loader to get train and test set ready for training
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=tr_sampl,num_workers=worker)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=te_sampl,num_workers=worker)
    return (trainloader, testloader)

In [3]:
dire = "./Data/Processed" # directory of dataset
# loading data loader
trainloader, testloader = preprocess_data(directory=dire, batch_size=16, test_size=0.3, rand_num=40, worker=4)
# getting device

In [4]:
# load pretrained model
# vgg16 = torchvision.models.vgg16(pretrained=True)
# googlenet = torchvision.models.googlenet(pretrained=True)
# efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [5]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained: nn.Module):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(2048, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, 5)
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [6]:
import train as trainer
# define params here
net = ClassifierNet(resnet50)
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
epochs = 1
trainer.train(net, optimizer, criterion, epochs, trainloader, testloader)

Training on cuda
tensor([[12.,  0.,  0.,  0., 13.],
        [10.,  0.,  0.,  0., 40.],
        [65.,  0.,  0.,  0., 68.],
        [10.,  0.,  0.,  0., 12.],
        [ 6.,  0.,  0.,  0., 19.]])


In [8]:
nb_classes = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(3, device='cuda:0') tensor(4, device='cuda:0')
tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(3, device='cuda:0') tensor(4, device='cuda:0')
tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(1, device='cuda:0') tensor(4, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(0, device='cuda:0') tensor(4, device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(2, device='cuda:0') tensor(0, device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0')
tensor(1, device='cuda:0') t